<a href="https://colab.research.google.com/github/asanam767/DRDO/blob/main/DRDO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install pandas numpy matplotlib seaborn tensorflow scikit-learn requests


In [5]:
import json
import requests
import pandas as pd

# Load the JSON metadata file
with open("7182799.json", "r") as f:
    data = json.load(f)

# Extract the links to CSV files
file_entries = data["files"]["entries"]

# Selecting files for participant U001
files_to_download = {
    "train_intention": file_entries["U001_train_intention.csv"]["links"]["content"],
    "train_payload": file_entries["U001_train_payload.csv"]["links"]["content"],
    "val_intention": file_entries["U001_val_intention.csv"]["links"]["content"],
    "val_payload": file_entries["U001_val_payload.csv"]["links"]["content"],
    "test_intention": file_entries["U001_test_intention.csv"]["links"]["content"],
    "test_payload": file_entries["U001_test_payload.csv"]["links"]["content"],
}

# Download and save the files
for name, url in files_to_download.items():
    response = requests.get(url)
    with open(f"{name}.csv", "wb") as f:
        f.write(response.content)
    print(f"Downloaded {name}.csv")


Downloaded train_intention.csv
Downloaded train_payload.csv
Downloaded val_intention.csv
Downloaded val_payload.csv
Downloaded test_intention.csv
Downloaded test_payload.csv


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!mv /content/train_intention.csv "/content/drive/My Drive/DRDO/"
!mv /content/train_payload.csv "/content/drive/My Drive/DRDO/"
!mv /content/test_intention.csv "/content/drive/My Drive/DRDO/"
!mv /content/test_payload.csv "/content/drive/My Drive/DRDO/"
!mv /content/val_intention.csv "/content/drive/My Drive/DRDO/"
!mv /content/val_payload.csv "/content/drive/My Drive/DRDO/"


In [8]:
import pandas as pd

# Load training, validation, and test datasets
df_train_intention = pd.read_csv("/content/drive/My Drive/DRDO/train_intention.csv")
df_train_payload = pd.read_csv("/content/drive/My Drive/DRDO/train_payload.csv")

df_val_intention = pd.read_csv("/content/drive/My Drive/DRDO/val_intention.csv")
df_val_payload = pd.read_csv("/content/drive/My Drive/DRDO/val_payload.csv")

df_test_intention = pd.read_csv("/content/drive/My Drive/DRDO/test_intention.csv")
df_test_payload = pd.read_csv("/content/drive/My Drive/DRDO/test_payload.csv")

# Display first 5 rows of each dataset
print("Train Intention Dataset:")
print(df_train_intention.head())

print("\nTrain Payload Dataset:")
print(df_train_payload.head())


Train Intention Dataset:
   Axel_X_C9  Axel_Y_C9  Axel_Z_C9  Gyro_X_C9  Gyro_Y_C9  Gyro_Z_C9  \
0   9.074494  -0.481113  -3.455022   0.012343   0.011154   0.046459   
1   9.234787  -0.546570  -3.346555   0.055380  -0.006489   0.026373   
2   9.308741  -0.685120  -3.125236   0.122433  -0.021939   0.019428   
3   9.189075  -0.693756  -2.841300   0.202340  -0.025395   0.003973   
4   9.190679  -0.460600  -2.688468   0.268421  -0.013397   0.001976   

   Axel_X_3B  Axel_Y_3B  Axel_Z_3B  Gyro_X_3B  ...  Gyro_X_B6  Gyro_Y_B6  \
0   9.294066   0.016188  -1.866136   0.280846  ...  -0.073917  -0.320395   
1   9.358950  -0.003909  -1.534498   0.243645  ...  -0.065672  -0.322257   
2   9.433356  -0.176038  -1.508867   0.232980  ...  -0.053523  -0.315645   
3   9.444161  -0.416103  -1.774503   0.247104  ...  -0.060104  -0.311796   
4   9.452289  -0.433332  -1.949570   0.219017  ...  -0.077943  -0.306367   

   Gyro_Z_B6  Axel_X_BB  Axel_Y_BB  Axel_Z_BB  Gyro_X_BB  Gyro_Y_BB  \
0  -0.011203   9.100

In [9]:
# Select all sensor columns (excluding the target column)
sensor_columns = df_train_intention.columns[:-1]  # Exclude 'target' column

# Extract features and labels
X_train_raw = df_train_intention[sensor_columns]
y_train_raw = df_train_intention['target']

X_val_raw = df_val_intention[sensor_columns]
y_val_raw = df_val_intention['target']

X_test_raw = df_test_intention[sensor_columns]
y_test_raw = df_test_intention['target']

print(f"Features Shape: {X_train_raw.shape}")
print(f"Labels Shape: {y_train_raw.shape}")


Features Shape: (109875, 30)
Labels Shape: (109875,)


In [10]:
from sklearn.preprocessing import LabelEncoder

# Check if labels are categorical (text-based)
if y_train_raw.dtype == 'O':  # 'O' means object (text data)
    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train_raw)
    y_val = encoder.transform(y_val_raw)
    y_test = encoder.transform(y_test_raw)

    print("Labels converted to numeric values.")
    print(f"Classes: {encoder.classes_}")
else:
    y_train, y_val, y_test = y_train_raw, y_val_raw, y_test_raw
    print("Labels are already numeric.")


Labels are already numeric.


In [11]:
import numpy as np

# Function to create sequences
def create_sequences(data, labels, seq_length=50):
    sequences = []
    sequence_labels = []

    for i in range(len(data) - seq_length):
        sequences.append(data.iloc[i : i + seq_length].values)  # 50 consecutive rows
        sequence_labels.append(labels.iloc[i + seq_length])  # Next row’s label

    return np.array(sequences), np.array(sequence_labels)

# Convert training data into sequences
X_train, y_train = create_sequences(X_train_raw, pd.Series(y_train_raw))
X_val, y_val = create_sequences(X_val_raw, pd.Series(y_val_raw))
X_test, y_test = create_sequences(X_test_raw, pd.Series(y_test_raw))

print("✅ Sequences created successfully!")
print(f"X_train shape: {X_train.shape}")  # Expected: (samples, 50, 30)
print(f"y_train shape: {y_train.shape}")  # Expected: (samples,)


✅ Sequences created successfully!
X_train shape: (109825, 50, 30)
y_train shape: (109825,)


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [13]:
# Define the LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),  # Dropout to reduce overfitting
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(len(set(y_train)), activation='softmax')  # Output layer for classification
])

# Display model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50, 64)              │          24,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │              68 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 37,332 (145.83 KB)

 Trainable params: 37,332 (145.83 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Compile the model
model.compile(
    loss='sparse_categorical_crossentropy',  # Multi-class classification loss
    optimizer='adam',  # Adaptive learning optimizer
    metrics=['accuracy']  # Track accuracy during training
)

print("✅ Model compiled successfully!")


✅ Model compiled successfully!


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=5,   # Start with 5 epochs
    batch_size=32,  # Efficient batch size for GPU
    validation_data=(X_val, y_val)
)


Epoch 1/5
2128/3433 ━━━━━━━━━━━━━━━━━━━━ 1:01 47ms/step - accuracy: 0.9025 - loss: 0.2492

In [4]:
model.save("/content/drive/My Drive/DRDO/lstm_model.keras")
print("✅ Model saved in the new Keras format!")


NameError: name 'model' is not defined

from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("/content/drive/My Drive/DRDO/lstm_model.h5")

print("✅ Model loaded successfully!")


In [ ]:
# Evaluate model on test dataset
loss, accuracy = model.evaluate(X_test, y_test)

print(f"✅ Test Accuracy: {accuracy * 100:.2f}%")
print(f"🔹 Test Loss: {loss:.4f}")
